# CREATE A SAMPLE DATASET

In [132]:
import pandas as pd
import numpy as np

# Define the days of the week
days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Generate random data for each column
weekday = np.random.choice(days_of_week, size=40000)
weather = np.random.choice(['Rainy', 'Sunny'], size=40000)
time_of_arrival = np.zeros(40000)

#Try with this sample, close to the observations of a 5 years old child, and see what happens :)
#weekday = np.random.choice(days_of_week, size=2000)
#weather = np.random.choice(['Rainy', 'Sunny'], size=2000)
#time_of_arrival = np.zeros(2000)

# Assign average time of arrival based on the patterns

###See that the time of arrival decrease from Monday to Thursday, but increase on Friday

for i in range(len(weekday)):
    if weekday[i] == 'Monday':
        time_of_arrival[i] = np.random.normal(loc=10.0, scale=1.0)
    elif weekday[i] == 'Tuesday':
        time_of_arrival[i] = np.random.normal(loc=9.5, scale=1.0)
    elif weekday[i] == 'Wednesday':
        time_of_arrival[i] = np.random.normal(loc=9.0, scale=1.0)
    elif weekday[i] == 'Thursday':
        time_of_arrival[i] = np.random.normal(loc=8.5, scale=1.0)
    elif weekday[i] == 'Friday':
        time_of_arrival[i] = np.random.normal(loc=5.0, scale=1.0)

###     # Increase time of arrival by 30% if the weather is rainy
        if weather[i] == 'Rainy':
            time_of_arrival[i] *= 1.3

# Create a DataFrame using the generated data
data = pd.DataFrame({'Day of the Week_TEXT': weekday,
                     'Weather_TEXT': weather,
                     'Time of Arrival': time_of_arrival})

# Display the first few rows of the dataset
print(data.head())


  Day of the Week_TEXT Weather_TEXT  Time of Arrival
0               Friday        Sunny         3.198904
1              Tuesday        Sunny         9.181509
2               Friday        Rainy         6.817752
3              Tuesday        Sunny        10.147400
4               Monday        Sunny         8.803722


# CREATE A NEURAL NETWORK TO PREVIEW TIME OF ARRIVAL

In [133]:
#import pandas as pd
#import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [134]:
encoder = LabelEncoder()
combined_labels = np.concatenate([data['Day of the Week_TEXT'].values, data['Weather_TEXT'].values])
encoder.fit(combined_labels)
data['Day of the Week'] = encoder.transform(data['Day of the Week_TEXT'])
data['Weather'] = encoder.transform(data['Weather_TEXT'])

#print(data.head())

  Day of the Week_TEXT Weather_TEXT  Time of Arrival  Day of the Week  Weather
0               Friday        Sunny         3.198904                0        3
1              Tuesday        Sunny         9.181509                5        3
2               Friday        Rainy         6.817752                0        2
3              Tuesday        Sunny        10.147400                5        3
4               Monday        Sunny         8.803722                1        3


In [135]:


# Load the dataset
#data = pd.read_csv('your_dataset.csv')  # Replace 'your_dataset.csv' with the actual dataset file name

# Preprocess the data
encoder = LabelEncoder()
combined_labels = np.concatenate([data['Day of the Week_TEXT'].values, data['Weather_TEXT'].values])
encoder.fit(combined_labels)
data['Day of the Week'] = encoder.transform(data['Day of the Week_TEXT'])
data['Weather'] = encoder.transform(data['Weather_TEXT'])

test_data = data.drop('Time of Arrival', axis=1)

columns_to_drop = ['Time of Arrival', 'Day of the Week_TEXT', 'Weather_TEXT']
X = data.drop(columns_to_drop, axis=1)
#X = data.drop('Time of Arrival', axis=1)
y = data['Time of Arrival']
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Create a DataFrame with merged data
test_data = pd.concat([test_data, pd.DataFrame(X)], axis=1)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))



Epoch 1/50
1000/1000 [==============================] - 1s 973us/step - loss: 7.3980 - val_loss: 2.1460
Epoch 2/50
1000/1000 [==============================] - 1s 919us/step - loss: 2.6340 - val_loss: 1.2392
Epoch 3/50
1000/1000 [==============================] - 1s 916us/step - loss: 2.2050 - val_loss: 1.2757
Epoch 4/50
1000/1000 [==============================] - 1s 914us/step - loss: 1.9990 - val_loss: 1.2512
Epoch 5/50
1000/1000 [==============================] - 1s 931us/step - loss: 1.8546 - val_loss: 1.1848
Epoch 6/50
1000/1000 [==============================] - 1s 938us/step - loss: 1.7736 - val_loss: 1.1652
Epoch 7/50
1000/1000 [==============================] - 1s 937us/step - loss: 1.7082 - val_loss: 1.1860
Epoch 8/50
1000/1000 [==============================] - 1s 931us/step - loss: 1.6513 - val_loss: 1.2353
Epoch 9/50
1000/1000 [==============================] - 1s 934us/step - loss: 1.6197 - val_loss: 1.1924
Epoch 10/50
1000/1000 [==============================] - 1s 945u

In [136]:
#Model Evaluate
model.evaluate(X_test,  y_test, verbose=2)

250/250 - 0s - loss: 1.1045 - 142ms/epoch - 566us/step


1.1045067310333252

In [137]:
test_data = test_data.drop_duplicates()
#print(test_data)

In [138]:
# Create a dataset with columns 'Day of the Week_TEXT' and 'Weather_TEXT'
label_dataset = test_data[['Day of the Week_TEXT', 'Weather_TEXT']]

# Create a dataset with the data from X (converted to a numpy array)
predict_dataset = test_data.drop(['Day of the Week_TEXT', 'Weather_TEXT','Day of the Week','Weather'], axis=1).to_numpy()
#print(predict_dataset)

In [139]:
# Make predictions on distinct values
predictions = model.predict(predict_dataset)

label_dataset_t = label_dataset.copy()

#print(len(predictions))
for i in range(len(predictions)):
    label_dataset_t.loc[label_dataset_t.index[i], 'Predictions'] = predictions[i]

# Display the resulting label_dataset
#print(label_dataset_t)

1/1 [==============================] - 0s 39ms/step


In [140]:
# Create a Categorical data type for 'Day of the Week_TEXT' column with the desired order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
label_dataset_t['Day of the Week_TEXT'] = pd.Categorical(label_dataset_t['Day of the Week_TEXT'], categories=day_order, ordered=True)

# Create a Categorical data type for 'Weather_TEXT' column with the desired order
weather_order = ['Sunny', 'Rainy']
label_dataset_t['Weather_TEXT'] = pd.Categorical(label_dataset_t['Weather_TEXT'], categories=weather_order, ordered=True)


# Sort the dataset by 'Day of the Week_TEXT' and 'Weather_TEXT'
sorted_dataset = label_dataset_t.sort_values(by=['Day of the Week_TEXT', 'Weather_TEXT'])

print(sorted_dataset)

   Day of the Week_TEXT Weather_TEXT  Predictions
4                Monday        Sunny     9.943331
14               Monday        Rainy     9.893540
1               Tuesday        Sunny     9.366679
6               Tuesday        Rainy     9.343451
12            Wednesday        Sunny     9.168489
16            Wednesday        Rainy     9.106167
15             Thursday        Sunny     8.593886
13             Thursday        Rainy     8.640964
0                Friday        Sunny     5.413193
2                Friday        Rainy     6.673346
